Let's load the data from the csv just as in `dataset.ipynb`.

In [1]:
import pandas as pd
import numpy as np

raw_data_file_name = "../dataset/fer2013.csv"
raw_data = pd.read_csv(raw_data_file_name)

Now, we separate and clean the data a little bit. First, we create an array of only the training data. Then, we create an array of only the private test data (referred to in the code with the prefix `first_test`). The `reset_index` call re-aligns the `first_test_data` to index from 0 instead of wherever it starts in the set.

In [2]:
train_data = raw_data[raw_data["Usage"] == "Training"]

first_test_data = raw_data[raw_data["Usage"] == "PrivateTest"]
first_test_data.reset_index(inplace=True)

In [3]:
import keras

train_expected = keras.utils.to_categorical(train_data["emotion"], num_classes=7, dtype='int32')
first_test_expected = keras.utils.to_categorical(first_test_data["emotion"], num_classes=7, dtype='int32')

Using TensorFlow backend.


In [4]:
def process_pixel_row(row):
    split = row.split(" ")
    pixels = np.array(split, 'float32')
    pixels /= 255
    return pixels.reshape(48, 48)

In [5]:
train_pixels = train_data["pixels"].apply(process_pixel_row)

In [6]:
first_test_pixels = first_test_data["pixels"].apply(process_pixel_row)

In [7]:
def duplicate_input_layer(array_input):
    vg_input = np.empty([int(len(array_input)), 48, 48, 3])
    for index, item in enumerate(vg_input):
        item[:, :, 0] = array_input[index]
        item[:, :, 1] = array_input[index]
        item[:, :, 2] = array_input[index]
    return vg_input

In [8]:
train_pixels_duplicated = duplicate_input_layer(train_pixels)

In [9]:
first_test_pixels_duplicated = duplicate_input_layer(first_test_pixels)

Now, let's load up a pre-trained neural network from `keras`, a high-level API on TensorFlow.

In [10]:
vgg16 = keras.applications.vgg16.VGG16(include_top=False, input_shape=(48, 48, 3), weights='imagenet', pooling='avr')

In [11]:
train_output_features = vgg16.predict(train_pixels_duplicated)

In [12]:
first_test_output_features = vgg16.predict(first_test_pixels_duplicated)

Here, we create our own top-level network to load on top of VGG16.

In [13]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Reshape

top_layer_model = Sequential()
top_layer_model.add(Reshape((512,), input_shape=(1, 1, 512)))
top_layer_model.add(Dense(256, input_shape=(512,), activation='relu'))
top_layer_model.add(Dense(256, input_shape=(256,), activation='relu'))
top_layer_model.add(Dropout(0.5))
top_layer_model.add(Dense(128, input_shape=(256,), activation='relu'))
top_layer_model.add(Dense(7, activation='softmax'))

In [14]:
from keras import optimizers

adamax = optimizers.Adamax()

top_layer_model.compile(loss='categorical_crossentropy',
              optimizer=adamax, metrics=['accuracy'])

top_layer_model.fit(train_output_features, train_expected,
          validation_data=(train_output_features, train_expected),
          epochs=20, batch_size=64)

Train on 28709 samples, validate on 28709 samples
Epoch 1/20
28709/28709 [==============================] - 2s 85us/step - loss: 1.6675 - accuracy: 0.3347 - val_loss: 1.5775 - val_accuracy: 0.3876
Epoch 2/20
28709/28709 [==============================] - 2s 81us/step - loss: 1.5816 - accuracy: 0.3852 - val_loss: 1.5401 - val_accuracy: 0.4029
Epoch 3/20
28709/28709 [==============================] - 2s 76us/step - loss: 1.5414 - accuracy: 0.4043 - val_loss: 1.4856 - val_accuracy: 0.4300
Epoch 4/20
28709/28709 [==============================] - 2s 79us/step - loss: 1.5089 - accuracy: 0.4219 - val_loss: 1.4617 - val_accuracy: 0.4407
Epoch 5/20
28709/28709 [==============================] - 2s 75us/step - loss: 1.4811 - accuracy: 0.4324 - val_loss: 1.4421 - val_accuracy: 0.4452
Epoch 6/20
28709/28709 [==============================] - 2s 78us/step - loss: 1.4496 - accuracy: 0.4440 - val_loss: 1.3911 - val_accuracy: 0.4711
Epoch 7/20
28709/28709 [==============================] - 2s 74us/st

In [15]:
score = top_layer_model.evaluate(first_test_output_features, first_test_expected, batch_size=32)

print("After top_layer_model training (test set): {}".format(score))

3589/3589 [==============================] - 0s 23us/step
After top_layer_model training (test set): [1.5379799089580566, 0.46865421533584595]


In [16]:
from keras.models import Model
from keras.layers import Input

inputs = Input(shape=(48, 48, 3))
vg_output = vgg16(inputs)

# TODO: the 'pooling' argument of the VGG16 model is important for this to work otherwise you will have to  squash
# output from (?, 1, 1, 512) to (?, 512)

model_predictions = top_layer_model(vg_output)
final_model = Model(inputs=inputs, outputs=model_predictions)
final_model.compile(loss='categorical_crossentropy',optimizer=adamax, metrics=['accuracy'])

In [17]:
final_model_score = final_model.evaluate(train_pixels_duplicated, train_expected, batch_size=32)
print("Sanity check - final_model (train score): {}".format(final_model_score))

28709/28709 [==============================] - 195s 7ms/step
Sanity check - final_model (train score): [0.9542508401144568, 0.6538715958595276]


In [18]:
final_model_score = final_model.evaluate(first_test_pixels_duplicated, first_test_expected, batch_size=32)
print("Sanity check - final_model (test score): {}".format(final_model_score))

3589/3589 [==============================] - 25s 7ms/step
Sanity check - final_model (test score): [1.5379799089580566, 0.46865421533584595]
